In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/MachineLearning/IDS-2017

/content/drive/MyDrive/MachineLearning/IDS-2017


In [ ]:
!pip install pytorch-lightning

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
import pytorch_lightning as pl
from dataset import MyDataModule
from pytorch_lightning.callbacks import EarlyStopping
from torch import nn
import time
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score


import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class RCNN(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=2, kernel_size=3, dropout=0.1):
        super(RCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_size, hidden_size, kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        for i in range(num_layers - 1):
            self.conv.add_module(f"conv_{i+1}", nn.Sequential(
                nn.Conv1d(hidden_size, hidden_size, kernel_size=kernel_size, padding=1),
                nn.ReLU(),
                nn.Dropout(dropout)
            ))
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        # Input shape: (batch_size, seq_len, input_size)
        x = x.unsqueeze(1) # Add a third dimension (1) at index 1
        x = x.permute(0, 2, 1)  # Shape: (batch_size, input_size, seq_len)
        x = self.conv(x)  # Shape: (batch_size, hidden_size, seq_len)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool1d(x, kernel_size=x.size()[2])  # Shape: (batch_size, hidden_size, 1)
        x = x.squeeze()  # Shape: (batch_size, hidden_size)
        x, _ = self.rnn(x.unsqueeze(0))  # Shape: (1, batch_size, hidden_size)
        x = x.squeeze()  # Shape: (batch_size, hidden_size)
        x = self.fc(x)  # Shape: (batch_size, num_classes)
        return x

    def compute_loss_and_metrics(self, batch, stage):
        x, y = batch
        x = x.to(self.device)
        y = y.to(self.device)
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y)
        metrics = {f"{stage}_loss": loss}
        for metric_name, metric_info in self.metrics.items():
            metric_fn = metric_info["function"]
            metric_kwargs = metric_info.get("kwargs", {})
            metric_value = metric_fn(y, y_pred.argmax(dim=1), **metric_kwargs).to(self.device)
            metrics[f"{stage}_{metric_name}"] = metric_value
        return metrics

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = nn.functional.cross_entropy(y_pred, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = nn.functional.cross_entropy(y_pred, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True)
        preds = torch.argmax(y_pred, dim=1)
        acc = torch.sum(preds == y) / float(len(y))
        self.log('val_acc', acc, on_step=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = nn.functional.cross_entropy(y_pred, y)
        self.log('test_loss', loss, on_step=True, on_epoch=True)
        preds = torch.argmax(y_pred, dim=1)
        acc = torch.sum(preds == y) / float(len(y))
        self.log('test_acc', acc, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}  


In [ ]:
# Instantiate the data module
data_module = MyDataModule(
    dataroot='MachineLearningCVE/',
    batch_size=32,
    seed=42,
    n_splits=5
)

In [ ]:
data_module.setup()

train_dataloader = data_module.train_dataloader()
val_dataloader = data_module.val_dataloader()
test_dataloader = data_module.test_dataloader()

# Extract input_dim and num_classes from the train_dataset
input_dim = data_module.train_dataset.X.shape[1]
num_classes = len(torch.unique(data_module.train_dataset.y))

[########################################] | 100% Completed | 32.52 s
there are 2830743 flow records with 79 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries
converted to numeric


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
hidden_dim = 128    # Number of neurons in each hidden layer
# num_layers = 2      # Number of hidden layers
lr = 1e-3           # Learning rate
reg = 1e-4          # L2 regularization (weight decay)
dropout_rate = 0.1  # Dropout rate for dropout layers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Instantiate the model
model = RCNN(input_dim, hidden_dim,num_classes)
model.to(device)
# Instantiate the logger
logger = pl.loggers.TensorBoardLogger('logs/', name='softmax_regression')

# Instantiate the trainer
trainer = pl.Trainer(max_epochs=200, logger=logger,precision=32,
    log_every_n_steps=10 )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=True, mode='min')
trainer = pl.Trainer(callbacks=[early_stop_callback])  # Add other trainer parameters as needed

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# Train the model
trainer.fit(model, train_dataloader, val_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type       | Params
------------------------------------
0 | conv | Sequential | 78.6 K
1 | rnn  | GRU        | 198 K 
2 | fc   | Linear     | 3.9 K 
------------------------------------
280 K     Trainable params
0         Non-trainable params
280 K     Total params
1.122     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved. New best score: 0.165


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.002 >= min_delta = 0.001. New best score: 0.135


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.007 >= min_delta = 0.001. New best score: 0.128


In [ ]:
# Train the model
trainer.fit(model, train_dataloader, val_dataloader)